# 0. Setting

In [1]:
import os
import statsmodels.api as sm     # install statsmodels if not installed
import statsmodels.formula.api as smf
import random
from tqdm import tqdm
import pandas as pd
import numpy as np
from datetime import datetime as dt
from datetime import timedelta
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math

np.random.seed(76019367)

import statsmodels.api as sm     # install statsmodels if not installed
import statsmodels.formula.api as smf

# 1. Data preparing

In [2]:
file_list = list(os.listdir('D:/Working_Place/new_uber/data/data_processed'))
fhvhv_list = []
for filename in file_list:
    if 'f' in filename:
        fhvhv_list.append(filename)
fhvhv_list

['f1902.parquet',
 'f1903.parquet',
 'f1904.parquet',
 'f1905.parquet',
 'f1906.parquet',
 'f1907.parquet',
 'f1908.parquet',
 'f1909.parquet',
 'f1910.parquet',
 'f1911.parquet',
 'f1912.parquet',
 'f2001.parquet',
 'f2002.parquet',
 'f2003.parquet',
 'f2004.parquet',
 'f2005.parquet',
 'f2006.parquet',
 'f2007.parquet',
 'f2008.parquet',
 'f2009.parquet',
 'f2010.parquet',
 'f2011.parquet',
 'f2012.parquet',
 'f2101.parquet',
 'f2102.parquet',
 'f2103.parquet',
 'f2104.parquet',
 'f2105.parquet',
 'f2106.parquet',
 'f2107.parquet',
 'f2108.parquet',
 'f2109.parquet',
 'f2110.parquet',
 'f2111.parquet',
 'f2112.parquet',
 'f2201.parquet',
 'f2202.parquet',
 'f2203.parquet',
 'f2204.parquet',
 'f2205.parquet',
 'f2206.parquet',
 'f2207.parquet',
 'f2208.parquet',
 'f2209.parquet',
 'f2210.parquet',
 'f2211.parquet',
 'f2212.parquet']

In [4]:
data_samples = []
for file in tqdm(fhvhv_list):
    each_data = pd.read_parquet('D:/Working_Place/new_uber/data/data_processed/'+file,
                               columns=['hvfhs_license_num','pickup_datetime',
                                       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_distance',
                                       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
                                       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
                                       'average_speed'])
    each_data_sampled = each_data.sample(frac=0.05)
    data_samples.append(each_data_sampled)
    del each_data
    del each_data_sampled

100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [08:23<00:00, 10.70s/it]


In [5]:
fdata_sample = pd.concat(data_samples)
del data_samples
fdata_sample.head()

,hvfhs_license_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_distance,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,average_speed
index,,,,,,,,,,,,,,,,
17771879,HV0005,2019-02-25 17:31:04,2019-02-25 17:57:28,122,219,6.9391,1560,23.71,0.0,0.59,2.10,0.00,0.0,0.0,17.78,16.013308
478958,HV0003,2019-02-01 16:51:32,2019-02-01 16:57:37,123,178,1.6422,366,6.71,0.0,0.17,0.60,0.00,0.0,0.0,5.39,16.152787
4466489,HV0005,2019-02-07 09:12:28,2019-02-07 09:31:08,180,124,4.8461,1079,12.94,0.0,0.32,1.15,0.00,0.0,0.0,12.54,16.168638
15742269,HV0003,2019-02-23 00:39:25,2019-02-23 01:16:29,230,25,14.1519,2224,24.30,0.0,0.63,2.24,2.75,0.0,0.0,33.32,22.907752
4578629,HV0003,2019-02-07 13:16:59,2019-02-07 13:41:31,238,237,4.8622,1473,18.98,0.0,0.48,1.70,2.75,0.0,0.0,15.66,11.883177


In [6]:
len(fdata_sample),fdata_sample.columns

(35941665,
 Index(['hvfhs_license_num', 'pickup_datetime', 'dropoff_datetime',
        'PULocationID', 'DOLocationID', 'trip_distance', 'trip_time',
        'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
        'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
        'average_speed'],
       dtype='object'))

# 2. Basic Analysis

# -1. hvfhs_license_number
HV0002 : Juno \
HV0003 : Uber \
HV0004 : Via \
HV0005 : Lyft

In [ ]:
fdata_sample_by_license_number = fdata_sample.groupby('hvfhs_license_num').agg({'trip_distance':['count','mean'],
                                                                               'trip_time':'mean',
                                                                               'tips':'mean',
                                                                              'driver_pay':'mean',
                                                                               'average_speed':'mean'})
fdata_sample_by_license_number

In [ ]:
fdata_sample_by_license_number['trip_distance'].plot(kind='pie',y='count')

In [8]:
uber_trip_sample = fdata_sample[fdata_sample.hvfhs_license_num=='HV0003']

# -2. Uber trip sample

In [ ]:
uber_trip_sample.head()

In [ ]:
uber_trip_sample.columns

In [9]:
uber_trip_sample['tip_rate']=uber_trip_sample.apply(lambda x: 100*x.tips/(x.base_passenger_fare+x.tolls+x.bcf+x.sales_tax+x.congestion_surcharge+x.airport_fee),axis=1)
uber_trip_sample

D:\Anaconda\envs\new_uber\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,hvfhs_license_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_distance,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,average_speed,tip_rate
index,,,,,,,,,,,,,,,,,
478958,HV0003,2019-02-01 16:51:32,2019-02-01 16:57:37,123,178,1.6422,366,6.71,0.0,0.17,0.60,0.00,0.0,0.00,5.39,16.152787,0.000000
15742269,HV0003,2019-02-23 00:39:25,2019-02-23 01:16:29,230,25,14.1519,2224,24.30,0.0,0.63,2.24,2.75,0.0,0.00,33.32,22.907752,0.000000
4578629,HV0003,2019-02-07 13:16:59,2019-02-07 13:41:31,238,237,4.8622,1473,18.98,0.0,0.48,1.70,2.75,0.0,0.00,15.66,11.883177,0.000000
13557508,HV0003,2019-02-19 22:59:44,2019-02-19 23:05:19,60,248,2.0769,335,3.86,0.0,0.10,0.34,0.00,0.0,0.00,5.39,22.318925,0.000000
6965178,HV0003,2019-02-10 11:57:24,2019-02-10 12:05:02,68,186,1.4329,458,9.41,0.0,0.24,0.84,2.75,0.0,5.00,10.39,11.262969,37.764350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8565974,HV0003,2022-12-13 20:57:55,2022-12-13 21:09:26,227,89,4.1216,691,14.25,0.0,0.43,1.26,0.00,0.0,0.00,9.27,21.472880,0.000000
8667811,HV0003,2022-12-13 23:26:02,2022-12-14 00:01:14,246,49,11.0446,2112,44.90,0.0,1.35,3.98,2.75,0.0,7.94,30.62,18.826023,14.986787
8009296,HV0003,2022-12-12 21:00:01,2022-12-12 21:05:21,206,206,1.6422,320,7.91,0.0,0.24,0.70,0.00,0.0,0.00,5.94,18.474750,0.000000


In [10]:
uber_trip_sample['is_tip'] = uber_trip_sample['tips'].map(lambda x: 0 if x<=0 else 100)
uber_trip_sample

D:\Anaconda\envs\new_uber\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,hvfhs_license_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_distance,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,average_speed,tip_rate,is_tip
index,,,,,,,,,,,,,,,,,,
478958,HV0003,2019-02-01 16:51:32,2019-02-01 16:57:37,123,178,1.6422,366,6.71,0.0,0.17,0.60,0.00,0.0,0.00,5.39,16.152787,0.000000,0
15742269,HV0003,2019-02-23 00:39:25,2019-02-23 01:16:29,230,25,14.1519,2224,24.30,0.0,0.63,2.24,2.75,0.0,0.00,33.32,22.907752,0.000000,0
4578629,HV0003,2019-02-07 13:16:59,2019-02-07 13:41:31,238,237,4.8622,1473,18.98,0.0,0.48,1.70,2.75,0.0,0.00,15.66,11.883177,0.000000,0
13557508,HV0003,2019-02-19 22:59:44,2019-02-19 23:05:19,60,248,2.0769,335,3.86,0.0,0.10,0.34,0.00,0.0,0.00,5.39,22.318925,0.000000,0
6965178,HV0003,2019-02-10 11:57:24,2019-02-10 12:05:02,68,186,1.4329,458,9.41,0.0,0.24,0.84,2.75,0.0,5.00,10.39,11.262969,37.764350,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8565974,HV0003,2022-12-13 20:57:55,2022-12-13 21:09:26,227,89,4.1216,691,14.25,0.0,0.43,1.26,0.00,0.0,0.00,9.27,21.472880,0.000000,0
8667811,HV0003,2022-12-13 23:26:02,2022-12-14 00:01:14,246,49,11.0446,2112,44.90,0.0,1.35,3.98,2.75,0.0,7.94,30.62,18.826023,14.986787,100
8009296,HV0003,2022-12-12 21:00:01,2022-12-12 21:05:21,206,206,1.6422,320,7.91,0.0,0.24,0.70,0.00,0.0,0.00,5.94,18.474750,0.000000,0


In [11]:
uber_daily = uber_trip_sample.groupby([uber_trip_sample.pickup_datetime.dt.year,
                                       uber_trip_sample.pickup_datetime.dt.month,
                                        uber_trip_sample.pickup_datetime.dt.day]).agg({'trip_distance':['count','mean'],
                                                                                      'trip_time':'mean',
                                                                                      'base_passenger_fare':'mean',
                                                                                      'tips':'mean',
                                                                                       'tip_rate':['min','max','mean'],
                                                                                       'is_tip':'mean',
                                                                                      'driver_pay':'mean',
                                                                                      'average_speed':'mean'})
uber_daily

trip_distance            \
                                                        count      mean   
pickup_datetime pickup_datetime pickup_datetime                           
2019            2               1                       27319  5.896432   
                                2                       27842  6.152482   
                                3                       23498  6.365884   
                                4                       18534  6.667523   
                                5                       17626  6.465769   
...                                                       ...       ...   
2022            12              27                      15418  6.520699   
                                28                      16447  6.806237   
                                29                      17433  6.868923   
                                30                      20001  7.103892   
                                31                      23127  6.305756   

                                                   trip_time  \
                                                        mean   
pickup_datetime pickup_datetime pickup_datetime                
2019            2               1                1083.057250   
                                2                1048.660549   
                                3                 981.773385   
                                4                1122.897324   
                                5                1087.482469   
...                                                      ...   
2022            12              27               1048.932741   
                                28               1108.919013   
                                29               1119.911547   
                                30               1153.347833   
                                31                964.990790   

                                                base_passenger_fare      tips  \
                                                               mean      mean   
pickup_datetime pickup_datetime pickup_datetime                                 
2019            2               1                         14.420390  0.343388   
                                2                         13.314968  0.311846   
                                3                         13.164466  0.299897   
                                4                         14.382206  0.356500   
                                5                         14.317151  0.346934   
...                                                             ...       ...   
2022            12              27                        20.718297  1.009737   
                                28                        20.868980  0.999051   
                                29                        21.010111  0.975387   
                                30                        21.483158  0.926846   
                                31                        20.688726  0.800757   

                                                tip_rate               \
                                                     min          max   
pickup_datetime pickup_datetime pickup_datetime                         
2019            2               1                    0.0  1818.181818   
                                2                    0.0   174.621653   
                                3                    0.0   202.020202   
                                4                    0.0   303.030303   
                                5                    0.0   357.142857   
...                                                  ...          ...   
2022            12              27                   0.0   188.323917   
                                28                   0.0   146.627566   
                                29                   0.0   200.000000   
                                30                   0.0   225.988701   
                

In [ ]:
uber_daily['trip_distance','count'].plot(figsize=(12,7))

In [ ]:
uber_daily['trip_distance','mean'].plot(figsize=(12,7))

In [ ]:
uber_daily[uber_daily['trip_distance','mean']>12]

In [ ]:
uber_daily['tips','mean'].plot(figsize=(12,7))

In [ ]:
uber_daily['tip_rate','mean'].plot(figsize=(12,7))

In [ ]:
uber_daily['is_tip','mean'].plot(figsize=(12,7))

## *with nonzero tips

In [12]:
uber_sample_with_tip = uber_trip_sample[uber_trip_sample.is_tip==100]
uber_sample_with_tip

,hvfhs_license_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_distance,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,average_speed,tip_rate,is_tip
index,,,,,,,,,,,,,,,,,,
6965178,HV0003,2019-02-10 11:57:24,2019-02-10 12:05:02,68,186,1.4329,458,9.41,0.00,0.24,0.84,2.75,0.0,5.00,10.39,11.262969,37.764350,100
3826120,HV0003,2019-02-06 09:27:27,2019-02-06 09:42:23,244,243,2.8980,897,4.82,0.00,0.12,0.43,0.00,0.0,2.00,7.46,11.630769,37.243948,100
9555599,HV0003,2019-02-14 09:11:57,2019-02-14 09:35:16,229,238,5.1681,1399,21.39,0.00,0.53,1.90,2.75,0.0,3.00,18.12,13.298899,11.290930,100
13362207,HV0003,2019-02-19 17:24:47,2019-02-19 17:56:42,144,50,6.9069,1914,21.23,0.00,0.54,1.93,2.75,0.0,3.00,23.65,12.991034,11.342155,100
13490045,HV0003,2019-02-19 20:18:30,2019-02-19 20:41:53,224,138,15.6170,1402,31.75,5.76,0.94,3.33,2.75,0.0,3.00,33.02,40.100713,6.737031,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12909896,HV0003,2022-12-20 06:18:31,2022-12-20 06:22:51,62,62,0.9016,260,8.41,0.00,0.25,0.75,0.00,0.0,3.00,9.35,12.483692,31.880978,100
11643482,HV0003,2022-12-17 23:54:07,2022-12-18 00:03:24,112,80,1.9320,557,16.02,0.00,0.48,1.42,0.00,0.0,5.00,14.09,12.486894,27.901786,100
14870808,HV0003,2022-12-23 13:51:57,2022-12-23 14:43:54,143,83,11.3827,3117,42.18,0.00,1.27,3.74,2.75,0.0,5.00,35.75,13.146526,10.012014,100


In [13]:
uber_daily_with_tip = uber_sample_with_tip.groupby([uber_sample_with_tip.pickup_datetime.dt.year,
                                                    uber_sample_with_tip.pickup_datetime.dt.month,
                                                    uber_sample_with_tip.pickup_datetime.dt.day]).agg({'trip_distance':['count','mean'],
                                                                                                      'trip_time':'mean',
                                                                                                      'base_passenger_fare':'mean',
                                                                                                      'tips':'mean',
                                                                                                       'tip_rate':['min','max','mean'],
                                                                                                       'driver_pay':'mean'})
uber_daily_with_tip

trip_distance            \
                                                        count      mean   
pickup_datetime pickup_datetime pickup_datetime                           
2019            2               1                        3242  6.484332   
                                2                        3239  6.417682   
                                3                        2548  6.828018   
                                4                        2069  7.641072   
                                5                        1971  7.398608   
...                                                       ...       ...   
2022            12              27                       3242  7.127850   
                                28                       3449  7.304975   
                                29                       3504  7.338113   
                                30                       3872  7.892385   
                                31                       4208  6.477836   

                                                   trip_time  \
                                                        mean   
pickup_datetime pickup_datetime pickup_datetime                
2019            2               1                1152.507403   
                                2                1084.203149   
                                3                1025.363030   
                                4                1225.280812   
                                5                1190.613902   
...                                                      ...   
2022            12              27               1172.301357   
                                28               1223.102059   
                                29               1229.435788   
                                30               1277.267304   
                                31                991.621435   

                                                base_passenger_fare      tips  \
                                                               mean      mean   
pickup_datetime pickup_datetime pickup_datetime                                 
2019            2               1                         17.755540  2.893587   
                                2                         15.537354  2.680584   
                                3                         15.838065  2.765687   
                                4                         18.423760  3.193514   
                                5                         18.317849  3.102516   
...                                                             ...       ...   
2022            12              27                        23.730654  4.802011   
                                28                        23.662952  4.764103   
                                29                        23.779004  4.852720   
                                30                        24.523892  4.787668   
                                31                        21.502393  4.400929   

                                                 tip_rate               \
                                                      min          max   
pickup_datetime pickup_datetime pickup_datetime                          
2019            2               1                0.023471  1818.181818   
                                2                0.098814   174.621653   
                                3                0.033058   202.020202   
                                4                1.664586   303.030303   
                                5                0.051335   357.142857   
...                                                   ...          ...   
2022            12              27               0.077220   188.323917   
                                28               0.025687   146.627566   
                                29               0.012376   200.000000   
                                30               0.016399   225.988701   
   

# -3. 스리슬쩍 주가랑 연결해보기

In [14]:
uber_stock = pd.read_csv('D:/Working_Place/new_uber/data/UBER.csv')
uber_stock.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-05-10,42.000000,45.000000,41.060001,41.570000,41.570000,186322500
1,2019-05-13,38.790001,39.240002,36.080002,37.099998,37.099998,79442400
2,2019-05-14,38.310001,39.959999,36.849998,39.959999,39.959999,46661100
3,2019-05-15,39.369999,41.880001,38.950001,41.290001,41.290001,36086100
4,2019-05-16,41.480000,44.060001,41.250000,43.000000,43.000000,38115500
5,2019-05-17,41.980000,43.290001,41.270000,41.910000,41.910000,20225700
6,2019-05-20,41.189999,41.680000,39.459999,41.590000,41.590000,29222300
7,2019-05-21,42.000000,42.240002,41.250000,41.500000,41.500000,10802900
8,2019-05-22,41.049999,41.279999,40.500000,41.250000,41.250000,9089500
9,2019-05-23,40.799999,41.090000,40.020000,40.470001,40.470001,11119900


In [ ]:
uber_daily.loc[2019,2,1]['tips','mean']

In [15]:
def str_to_date(date):
    year = int(date[0:4])
    month = int(date[5:7])
    day = int(date[8:])
    return year,month,day
print(str_to_date('2019-05-10'))

(2019, 5, 10)


In [ ]:
uber_stock

In [16]:
def share_outstanding(date):
    if date<='2019-06-30':
        return 1.111*(10**9)
    elif date<='2019-09-30':
        return 1.700*(10**9)
    elif date<='2019-12-31':
        return 1.248*(10**9)
    elif date<='2020-03-31':
        return 1.724*(10**9)
    elif date<='2020-06-30':
        return 1.739*(10**9)
    elif date<='2020-09-30':
        return 1.755*(10**9)
    elif date<='2020-12-31':
        return 1.753*(10**9)
    elif date<='2021-03-31':
        return 1.859*(10**9)
    elif date<='2021-06-30':
        return 1.956*(10**9)
    elif date<='2021-09-30':
        return 1.899*(10**9)
    elif date<='2021-12-31':
        return 1.896*(10**9)
    elif date<='2022-03-31':
        return 1.958*(10**9)
    elif date<='2022-06-30':
        return 1.969*(10**9)
    elif date<='2022-09-30':
        return 1.979*(10**9)
    else:
        return 1.975*(10**9)

In [17]:
uber_stock['share_outstanding']=uber_stock['Date'].map(lambda x: share_outstanding(x))
uber_stock['mve'] = uber_stock['share_outstanding']*uber_stock['Close']
uber_stock

,Date,Open,High,Low,Close,Adj Close,Volume,share_outstanding,mve
0,2019-05-10,42.000000,45.000000,41.060001,41.570000,41.570000,186322500,1.111000e+09,4.618427e+10
1,2019-05-13,38.790001,39.240002,36.080002,37.099998,37.099998,79442400,1.111000e+09,4.121810e+10
2,2019-05-14,38.310001,39.959999,36.849998,39.959999,39.959999,46661100,1.111000e+09,4.439556e+10
3,2019-05-15,39.369999,41.880001,38.950001,41.290001,41.290001,36086100,1.111000e+09,4.587319e+10
4,2019-05-16,41.480000,44.060001,41.250000,43.000000,43.000000,38115500,1.111000e+09,4.777300e+10
...,...,...,...,...,...,...,...,...,...
1120,2023-10-20,42.720001,43.349998,42.259998,42.959999,42.959999,13941300,1.975000e+09,8.484600e+10
1121,2023-10-23,42.529999,43.970001,42.250000,43.040001,43.040001,11277100,1.975000e+09,8.500400e+10
1122,2023-10-24,43.500000,44.325001,43.384998,44.189999,44.189999,17566500,1.975000e+09,8.727525e+10
1123,2023-10-25,43.299999,43.490002,42.064999,42.349998,42.349998,16495900,1.975000e+09,8.364125e+10


In [18]:
def bva(date):
    if date<='2019-06-30':
        return 30.98*(10**9)
    elif date<='2019-09-30':
        return 32.29*(10**9)
    elif date<='2019-12-31':
        return 31.76*(10**9)
    elif date<='2020-03-31':
        return 30.09*(10**9)
    elif date<='2020-06-30':
        return 28.24*(10**9)
    elif date<='2020-09-30':
        return 28.89*(10**9)
    elif date<='2020-12-31':
        return 33.25*(10**9)
    elif date<='2021-03-31':
        return 34.66*(10**9)
    elif date<='2021-06-30':
        return 36.25*(10**9)
    elif date<='2021-09-30':
        return 36.88*(10**9)
    elif date<='2021-12-31':
        return 38.77*(10**9)
    elif date<='2022-03-31':
        return 32.81*(10**9)
    elif date<='2022-06-30':
        return 31.01*(10**9)
    elif date<='2022-09-30':
        return 31.11*(10**9)
    else:
        return 32.11*(10**9)

In [19]:
uber_stock['bva'] = uber_stock.apply(lambda x: bva(x.Date),axis=1)
uber_stock

,Date,Open,High,Low,Close,Adj Close,Volume,share_outstanding,mve,bva
0,2019-05-10,42.000000,45.000000,41.060001,41.570000,41.570000,186322500,1.111000e+09,4.618427e+10,3.098000e+10
1,2019-05-13,38.790001,39.240002,36.080002,37.099998,37.099998,79442400,1.111000e+09,4.121810e+10,3.098000e+10
2,2019-05-14,38.310001,39.959999,36.849998,39.959999,39.959999,46661100,1.111000e+09,4.439556e+10,3.098000e+10
3,2019-05-15,39.369999,41.880001,38.950001,41.290001,41.290001,36086100,1.111000e+09,4.587319e+10,3.098000e+10
4,2019-05-16,41.480000,44.060001,41.250000,43.000000,43.000000,38115500,1.111000e+09,4.777300e+10,3.098000e+10
...,...,...,...,...,...,...,...,...,...,...
1120,2023-10-20,42.720001,43.349998,42.259998,42.959999,42.959999,13941300,1.975000e+09,8.484600e+10,3.211000e+10
1121,2023-10-23,42.529999,43.970001,42.250000,43.040001,43.040001,11277100,1.975000e+09,8.500400e+10,3.211000e+10
1122,2023-10-24,43.500000,44.325001,43.384998,44.189999,44.189999,17566500,1.975000e+09,8.727525e+10,3.211000e+10
1123,2023-10-25,43.299999,43.490002,42.064999,42.349998,42.349998,16495900,1.975000e+09,8.364125e+10,3.211000e+10


In [20]:
def bvl(date):
    if date<='2019-06-30':
        return 15.058*(10**9)
    elif date<='2019-09-30':
        return 16.550*(10**9)
    elif date<='2019-12-31':
        return 16.889*(10**9)
    elif date<='2020-03-31':
        return 18.062*(10**9)
    elif date<='2020-06-30':
        return 18.001*(10**9)
    elif date<='2020-09-30':
        return 19.261*(10**9)
    elif date<='2020-12-31':
        return 20.285*(10**9)
    elif date<='2021-03-31':
        return 21.057*(10**9)
    elif date<='2021-06-30':
        return 22.076*(10**9)
    elif date<='2021-09-30':
        return 22.828*(10**9)
    elif date<='2021-12-31':
        return 23.629*(10**9)
    elif date<='2022-03-31':
        return 23.199*(10**9)
    elif date<='2022-06-30':
        return 23.645*(10**9)
    elif date<='2022-09-30':
        return 24.142*(10**9)
    else:
        return 24.035*(10**9)

In [21]:
uber_stock['bvl'] = uber_stock.apply(lambda x: bvl(x.Date),axis=1)
uber_stock

,Date,Open,High,Low,Close,Adj Close,Volume,share_outstanding,mve,bva,bvl
0,2019-05-10,42.000000,45.000000,41.060001,41.570000,41.570000,186322500,1.111000e+09,4.618427e+10,3.098000e+10,1.505800e+10
1,2019-05-13,38.790001,39.240002,36.080002,37.099998,37.099998,79442400,1.111000e+09,4.121810e+10,3.098000e+10,1.505800e+10
2,2019-05-14,38.310001,39.959999,36.849998,39.959999,39.959999,46661100,1.111000e+09,4.439556e+10,3.098000e+10,1.505800e+10
3,2019-05-15,39.369999,41.880001,38.950001,41.290001,41.290001,36086100,1.111000e+09,4.587319e+10,3.098000e+10,1.505800e+10
4,2019-05-16,41.480000,44.060001,41.250000,43.000000,43.000000,38115500,1.111000e+09,4.777300e+10,3.098000e+10,1.505800e+10
...,...,...,...,...,...,...,...,...,...,...,...
1120,2023-10-20,42.720001,43.349998,42.259998,42.959999,42.959999,13941300,1.975000e+09,8.484600e+10,3.211000e+10,2.403500e+10
1121,2023-10-23,42.529999,43.970001,42.250000,43.040001,43.040001,11277100,1.975000e+09,8.500400e+10,3.211000e+10,2.403500e+10
1122,2023-10-24,43.500000,44.325001,43.384998,44.189999,44.189999,17566500,1.975000e+09,8.727525e+10,3.211000e+10,2.403500e+10
1123,2023-10-25,43.299999,43.490002,42.064999,42.349998,42.349998,16495900,1.975000e+09,8.364125e+10,3.211000e+10,2.403500e+10


In [22]:
uber_stock_under_2023 = uber_stock[uber_stock.Date<'2023']
uber_stock_under_2023

,Date,Open,High,Low,Close,Adj Close,Volume,share_outstanding,mve,bva,bvl
0,2019-05-10,42.000000,45.000000,41.060001,41.570000,41.570000,186322500,1.111000e+09,4.618427e+10,3.098000e+10,1.505800e+10
1,2019-05-13,38.790001,39.240002,36.080002,37.099998,37.099998,79442400,1.111000e+09,4.121810e+10,3.098000e+10,1.505800e+10
2,2019-05-14,38.310001,39.959999,36.849998,39.959999,39.959999,46661100,1.111000e+09,4.439556e+10,3.098000e+10,1.505800e+10
3,2019-05-15,39.369999,41.880001,38.950001,41.290001,41.290001,36086100,1.111000e+09,4.587319e+10,3.098000e+10,1.505800e+10
4,2019-05-16,41.480000,44.060001,41.250000,43.000000,43.000000,38115500,1.111000e+09,4.777300e+10,3.098000e+10,1.505800e+10
...,...,...,...,...,...,...,...,...,...,...,...
914,2022-12-23,24.459999,24.725000,24.162001,24.639999,24.639999,11610000,1.975000e+09,4.866400e+10,3.211000e+10,2.403500e+10
915,2022-12-27,24.290001,24.700001,23.900000,24.400000,24.400000,15258600,1.975000e+09,4.819000e+10,3.211000e+10,2.403500e+10
916,2022-12-28,24.209999,24.719999,24.020000,24.590000,24.590000,15669200,1.975000e+09,4.856525e+10,3.211000e+10,2.403500e+10
917,2022-12-29,24.750000,25.375000,24.290001,24.910000,24.910000,17828300,1.975000e+09,4.919725e+10,3.211000e+10,2.403500e+10


In [23]:
tip_as_input = []
for i in uber_stock_under_2023.Date:
    year,month,day = str_to_date(i)
    tip_as_input.append(uber_daily.loc[year,month,day]['tips','mean'])
uber_stock_under_2023['tip']=tip_as_input

tip_rate_as_input = []
for i in uber_stock_under_2023.Date:
    year,month,day = str_to_date(i)
    tip_rate_as_input.append(uber_daily.loc[year,month,day]['tip_rate','mean'])
uber_stock_under_2023['tip_rate']=tip_rate_as_input

is_tip_as_input = []
for i in uber_stock_under_2023.Date:
    year,month,day = str_to_date(i)
    is_tip_as_input.append(uber_daily.loc[year,month,day]['is_tip','mean'])
uber_stock_under_2023['is_tip']=is_tip_as_input

uber_stock_under_2023

D:\Anaconda\envs\new_uber\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda\envs\new_uber\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\Anaconda\envs\new_uber\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,Date,Open,High,Low,Close,Adj Close,Volume,share_outstanding,mve,bva,bvl,tip,tip_rate,is_tip
0,2019-05-10,42.000000,45.000000,41.060001,41.570000,41.570000,186322500,1.111000e+09,4.618427e+10,3.098000e+10,1.505800e+10,0.458293,2.168560,11.749034
1,2019-05-13,38.790001,39.240002,36.080002,37.099998,37.099998,79442400,1.111000e+09,4.121810e+10,3.098000e+10,1.505800e+10,0.498037,2.228458,11.973866
2,2019-05-14,38.310001,39.959999,36.849998,39.959999,39.959999,46661100,1.111000e+09,4.439556e+10,3.098000e+10,1.505800e+10,0.496780,2.235511,12.195553
3,2019-05-15,39.369999,41.880001,38.950001,41.290001,41.290001,36086100,1.111000e+09,4.587319e+10,3.098000e+10,1.505800e+10,0.530976,2.286350,12.306853
4,2019-05-16,41.480000,44.060001,41.250000,43.000000,43.000000,38115500,1.111000e+09,4.777300e+10,3.098000e+10,1.505800e+10,0.549744,2.152365,11.936994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,2022-12-23,24.459999,24.725000,24.162001,24.639999,24.639999,11610000,1.975000e+09,4.866400e+10,3.211000e+10,2.403500e+10,0.976130,3.675002,19.403451
915,2022-12-27,24.290001,24.700001,23.900000,24.400000,24.400000,15258600,1.975000e+09,4.819000e+10,3.211000e+10,2.403500e+10,1.009737,3.798689,21.027371
916,2022-12-28,24.209999,24.719999,24.020000,24.590000,24.590000,15669200,1.975000e+09,4.856525e+10,3.211000e+10,2.403500e+10,0.999051,3.801013,20.970390
917,2022-12-29,24.750000,25.375000,24.290001,24.910000,24.910000,17828300,1.975000e+09,4.919725e+10,3.211000e+10,2.403500e+10,0.975387,3.709554,20.099811


In [24]:
uber_stock_under_2023['year'] = uber_stock_under_2023['Date'].map(lambda x: str_to_date(x)[0])
uber_stock_under_2023['month'] = uber_stock_under_2023['Date'].map(lambda x: str_to_date(x)[1])
uber_stock_under_2023['day'] = uber_stock_under_2023['Date'].map(lambda x: str_to_date(x)[2])

D:\Anaconda\envs\new_uber\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda\envs\new_uber\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\envs\new_uber\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [ ]:
def quarter(month):
    if 1<=month<=3:
        return 0
    elif month<=6:
        return 1
    elif month<=9:
        return 2
    else:
        return 3

In [ ]:
uber_stock_under_2023['quarter']=uber_stock_under_2023['month'].map(lambda x:quarter(x))

In [25]:
uber_stock_under_2023['ln_mve'] = np.log(uber_stock_under_2023.mve)
uber_stock_under_2023['ln_bva'] = np.log(uber_stock_under_2023.bva)
uber_stock_under_2023['ln_bvl'] = np.log(uber_stock_under_2023.bvl)

D:\Anaconda\envs\new_uber\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda\envs\new_uber\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\envs\new_uber\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [26]:
uber_stock_under_2023[(uber_stock_under_2023.tip<=0) |
                     (uber_stock_under_2023.tip_rate<=0)|
                     (uber_stock_under_2023.is_tip<=0)]

,Date,Open,High,Low,Close,Adj Close,Volume,share_outstanding,mve,bva,bvl,tip,tip_rate,is_tip,year,month,day,ln_mve,ln_bva,ln_bvl


In [ ]:
min(uber_stock_under_2023.tip[uber_stock_under_2023.tip>0]),min(uber_stock_under_2023.tip_rate[uber_stock_under_2023.tip_rate>0]),min(uber_stock_under_2023.is_tip[uber_stock_under_2023.is_tip>0])

In [27]:
uber_stock_under_2023['ln_tip'] = np.log(uber_stock_under_2023.tip)
uber_stock_under_2023['ln_tip_rate'] = np.log(uber_stock_under_2023.tip_rate)
uber_stock_under_2023['ln_is_tip'] = np.log(uber_stock_under_2023.is_tip)

D:\Anaconda\envs\new_uber\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda\envs\new_uber\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda\envs\new_uber\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [ ]:
uber_stock_under_2023.head()

## 0) default

In [28]:
zeroth_regression = smf.ols("ln_mve ~ ln_bva + ln_bvl + C(year)", 
                           data=uber_stock_under_2023).fit()
print(zeroth_regression.summary())

                            OLS Regression Results                            
Dep. Variable:                 ln_mve   R-squared:                       0.658
Model:                            OLS   Adj. R-squared:                  0.657
Method:                 Least Squares   F-statistic:                     351.9
Date:                Thu, 02 Nov 2023   Prob (F-statistic):          4.48e-210
Time:                        17:25:28   Log-Likelihood:                 285.78
No. Observations:                 919   AIC:                            -559.6
Df Residuals:                     913   BIC:                            -530.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           6.7671      3.751     

## 1) just tip

In [29]:
first_regression = smf.ols("mve ~ bva + bvl + tip", 
                           data=uber_stock_under_2023).fit()
print(first_regression.summary())

                            OLS Regression Results                            
Dep. Variable:                    mve   R-squared:                       0.676
Model:                            OLS   Adj. R-squared:                  0.675
Method:                 Least Squares   F-statistic:                     635.8
Date:                Thu, 02 Nov 2023   Prob (F-statistic):          3.07e-223
Time:                        17:25:36   Log-Likelihood:                -22599.
No. Observations:                 919   AIC:                         4.521e+04
Df Residuals:                     915   BIC:                         4.523e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.248e+11   4.66e+09    -26.797      0.0

In [30]:
first_regression = smf.ols("mve ~ bva + bvl + tip + C(year)", 
                           data=uber_stock_under_2023).fit()
print(first_regression.summary())

                            OLS Regression Results                            
Dep. Variable:                    mve   R-squared:                       0.752
Model:                            OLS   Adj. R-squared:                  0.751
Method:                 Least Squares   F-statistic:                     461.7
Date:                Thu, 02 Nov 2023   Prob (F-statistic):          2.50e-272
Time:                        17:25:41   Log-Likelihood:                -22476.
No. Observations:                 919   AIC:                         4.497e+04
Df Residuals:                     912   BIC:                         4.500e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept       -1.565e+10    8.4e+09     

In [ ]:
second_regression = smf.ols("ln_mve ~ ln_bva + ln_bvl + ln_tip", 
                           data=uber_stock_under_2023).fit()
print(second_regression.summary())

In [31]:
second_regression = smf.ols("ln_mve ~ ln_bva + ln_bvl + ln_tip+C(year)", 
                           data=uber_stock_under_2023).fit()
print(second_regression.summary())

                            OLS Regression Results                            
Dep. Variable:                 ln_mve   R-squared:                       0.694
Model:                            OLS   Adj. R-squared:                  0.692
Method:                 Least Squares   F-statistic:                     344.8
Date:                Thu, 02 Nov 2023   Prob (F-statistic):          1.41e-230
Time:                        17:25:50   Log-Likelihood:                 336.47
No. Observations:                 919   AIC:                            -658.9
Df Residuals:                     912   BIC:                            -625.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -8.5547      3.850     

In [ ]:
second_regression = smf.ols("ln_mve ~ ln_bva + ln_bvl + ln_tip+ln_trip_amount+C(year)", 
                           data=uber_stock_under_2023).fit()
print(second_regression.summary())

## 2) tip rate

In [ ]:
tip_rate_regression = smf.ols("mve ~ bva + bvl + tip_rate", 
                           data=uber_stock_under_2023).fit()
print(tip_rate_regression.summary())

In [32]:
tip_rate_regression = smf.ols("mve ~ bva + bvl + tip_rate+C(year)", 
                           data=uber_stock_under_2023).fit()
print(tip_rate_regression.summary())

                            OLS Regression Results                            
Dep. Variable:                    mve   R-squared:                       0.759
Model:                            OLS   Adj. R-squared:                  0.758
Method:                 Least Squares   F-statistic:                     479.2
Date:                Thu, 02 Nov 2023   Prob (F-statistic):          6.87e-278
Time:                        17:25:54   Log-Likelihood:                -22463.
No. Observations:                 919   AIC:                         4.494e+04
Df Residuals:                     912   BIC:                         4.497e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept       -6.966e+09   8.16e+09     

In [33]:
ln_tip_rate_regression = smf.ols("ln_mve ~ ln_bva + ln_bvl + ln_tip_rate+C(year)", 
                           data=uber_stock_under_2023).fit()
print(ln_tip_rate_regression.summary())

                            OLS Regression Results                            
Dep. Variable:                 ln_mve   R-squared:                       0.696
Model:                            OLS   Adj. R-squared:                  0.694
Method:                 Least Squares   F-statistic:                     347.6
Date:                Thu, 02 Nov 2023   Prob (F-statistic):          1.15e-231
Time:                        17:25:59   Log-Likelihood:                 339.00
No. Observations:                 919   AIC:                            -664.0
Df Residuals:                     912   BIC:                            -630.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -7.4941      3.790     

In [ ]:
corr_table = uber_stock_under_2023[['ln_mve','ln_bva','ln_bvl','ln_tip_rate','ln_trip_amount']].corr()
plt.rcParams['figure.figsize']=(5,5)
sns.heatmap(corr_table, annot=True)

In [ ]:
ln_tip_rate_regression_with_date = smf.ols("ln_mve ~ ln_bva + ln_bvl + ln_tip_rate + C(year)", 
                                           data=uber_stock_under_2023).fit()
print(ln_tip_rate_regression_with_date.summary())

In [ ]:
ln_tip_rate_ln_trip_amount_regression_with_date = smf.ols("ln_mve ~ ln_bva + ln_bvl + ln_tip_rate + ln_trip_amount + C(year)", 
                                                           data=uber_stock_under_2023).fit()
print(ln_tip_rate_ln_trip_amount_regression_with_date.summary())

## 3) is_tip

In [ ]:
is_tip_regression = smf.ols("mve ~ bva + bvl + is_tip", 
                            data=uber_stock_under_2023).fit()
print(is_tip_regression.summary())

In [ ]:
is_tip_regression_with_date = smf.ols("mve ~ bva + bvl + is_tip + C(year)", 
                            data=uber_stock_under_2023).fit()
print(is_tip_regression_with_date.summary())

In [ ]:
ln_is_tip_regression_with_date = smf.ols("ln_mve ~ ln_bva + ln_bvl + ln_is_tip + C(year)", 
                            data=uber_stock_under_2023).fit()
print(ln_is_tip_regression_with_date.summary())

## 4) How about COVID-19 effect?

In [ ]:
uber_stock_under_2023['covid19']=uber_stock_under_2023.Date.map(lambda x: 1 if x>='2020-03-13' else 0)
uber_stock_under_2023.head()

In [ ]:
ln_is_tip_regression_with_date_covid19 = smf.ols("ln_mve ~ ln_bva + ln_bvl + ln_is_tip + ln_trip_amount+ C(year) +C(covid19)", 
                            data=uber_stock_under_2023).fit()
print(ln_is_tip_regression_with_date_covid19.summary())

## 5) How about nonzero tips?

In [ ]:
tip_as_input = []
for i in uber_stock_under_2023.Date:
    year,month,day = str_to_date(i)
    if year==2019 and month==7 and day==31:
        tip_as_input.append(min(tip_as_input))
    else:
        tip_as_input.append(uber_daily_with_tip.loc[year,month,day]['tips','mean'])
uber_stock_under_2023['nonzero_tip']=tip_as_input

tip_rate_as_input = []
for i in uber_stock_under_2023.Date:
    year,month,day = str_to_date(i)
    if year==2019 and month==7 and day==31:
        tip_rate_as_input.append(min(tip_rate_as_input))
    else:
        tip_rate_as_input.append(uber_daily_with_tip.loc[year,month,day]['tip_rate','mean'])
uber_stock_under_2023['nonzero_tip_rate']=tip_rate_as_input

uber_stock_under_2023

In [ ]:
uber_stock_under_2023['ln_nonzero_tip'] = np.log(uber_stock_under_2023.nonzero_tip)
uber_stock_under_2023['ln_nonzero_tip_rate'] = np.log(uber_stock_under_2023.nonzero_tip_rate)


In [ ]:
ln_nonzero_tip_rate_regression_with_date = smf.ols("ln_mve ~ ln_bva + ln_bvl + ln_nonzero_tip_rate + C(year)", 
                                           data=uber_stock_under_2023).fit()
print(ln_nonzero_tip_rate_regression_with_date.summary())

## 6) tip_rate over trip distance 

# 3. Graphs

In [ ]:
uber_trip_sample

In [ ]:
uber_trip_sample.columns

# -1. by Time

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.year).agg({'trip_distance':['mean','var']})[('trip_distance','mean')].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.month).agg({'trip_distance':['mean','var']})[('trip_distance','mean')].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.weekday).agg({'trip_distance':['mean','var']})[('trip_distance','mean')].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.year).agg({'trip_time':['mean','var']})[('trip_time','mean')].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.month).agg({'trip_time':['mean','var']})[('trip_time','mean')].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.weekday).agg({'trip_time':['mean','var']})[('trip_time','mean')].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.year).agg({'tips':['mean','var']})[('tips','mean')].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.month).agg({'tips':['mean','var']})[('tips','mean')].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.weekday).agg({'tips':['mean','var']})[('tips','mean')].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.hour).agg({'tips':['mean','var']})[('tips','mean')].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.year).agg({'tip_rate':['mean','var']})[('tip_rate','mean')].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.month).agg({'tip_rate':['mean','var']})[('tip_rate','mean')].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.weekday).agg({'tip_rate':['mean','var']})[('tip_rate','mean')].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.hour).agg({'tip_rate':['mean','var']})[('tip_rate','mean')].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.year).agg({'is_tip':['sum','mean']})['is_tip','mean'].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.month).agg({'is_tip':['sum','mean']})['is_tip','mean'].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.weekday).agg({'is_tip':['mean','sum']})[('is_tip','mean')].plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.pickup_datetime.dt.hour).agg({'is_tip':['mean','var']})[('is_tip','mean')].plot(kind='bar')

# -2. Location

In [ ]:
taxi_zone = pd.read_csv('D:/Working_Place/new_uber/data/taxi_zone_lookup.csv')
taxi_zone

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.PULocationID).agg({'tip_rate':'mean'}).plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.PULocationID).agg({'is_tip':'mean'}).plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.DOLocationID).agg({'tip_rate':'mean'}).plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.DOLocationID).agg({'is_tip':'mean'}).plot(kind='bar')

In [ ]:
uber_trip_sample.groupby(uber_trip_sample.DOLocationID).agg({'is_tip':'mean'})[uber_trip_sample.groupby(uber_trip_sample.DOLocationID).agg({'is_tip':'mean'}).is_tip>0.3]

In [ ]:
taxi_zone[(taxi_zone.LocationID==110) | (taxi_zone.LocationID==138)]

# 4. Regression

In [ ]:
uber_trip_sample

In [ ]:
uber_trip_sample.columns

In [ ]:
corr_dt=uber_trip_sample[['tips','tip_rate','is_tip','trip_distance','trip_time','base_passenger_fare','congestion_surcharge','airport_fee','average_speed']].corr()
plt.rcParams['figure.figsize']=(10,10)
sns.heatmap(corr_dt, annot=True)

In [ ]:
zeroth_regression = smf.ols("tips ~ trip_time + average_speed+\
                            C(pickup_datetime.dt.year)+\
                            C(pickup_datetime.dt.month)+\
                            C(pickup_datetime.dt.day)+\", 
                           data=uber_trip_sample).fit()
print(zeroth_regression.summary())